# General Settings and Imports

In [336]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [337]:
#general stuff
import pandas as pd
import numpy as np
import json
from pprint import pprint

#ofac stuff
import csv
import requests
import json
import xmltodict
from collections import defaultdict

#graphsense stuff
import graphsense
from graphsense.api import addresses_api, bulk_api, entities_api, general_api

from graphsense.model.neighbor_addresses import NeighborAddresses

# Defining the host is optional and defaults to https://api.ikna.io
# See configuration.py for a list of all supported configuration parameters.
#configuration = graphsense.Configuration(
    #host = "https://api.ikna.io"
#)

# The client must configure the authentication and authorization parameters
# in accordance with the API server security policy.
# Examples for each auth method are provided below, use the example that
# satisfies your auth use case.

# Configure API key authorization: api_key
#configuration.api_key['api_key'] = 'YOUR_API_KEY'

In [338]:
f = open('config.json')
config = json.load(f)
f.close()

CURRENCY = 'eth'
ADDRESS = '0x49bd56B275395130B41D67E1304b2d49F8A88725'

configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

#with graphsense.ApiClient(configuration) as api_client:
    #api_instance = general_api.GeneralApi(api_client)
    #api_response = api_instance.get_statistics()
    #pprint(api_response)

# Associated Wallets Module

In [339]:
class AssociatedWallets():

    def __init__(self, currency):
        self.CURRENCY = currency
    
    def simple_neighbors(self, address, direction):
        with graphsense.ApiClient(configuration) as api_client:
            # Create an instance of the API class
            api_instance = addresses_api.AddressesApi(api_client)
            currency = self.CURRENCY # str | The cryptocurrency code (e.g., eth)
            address = ADDRESS # str | The cryptocurrency address
            direction = "out" # str | Incoming or outgoing neighbors
            only_ids = [
                "only_ids_example",
            ] # [str] | Restrict result to given set of comma separated addresses (optional)
            include_labels = False # bool | Whether to include labels of first page of address tags (optional) if omitted the server will use the default value of False
            page = "page_example" # str | Resumption token for retrieving the next page (optional)
            pagesize = 10 # int | Number of items returned in a single page (optional)

            # example passing only required values which don't have defaults set
            try:
                # Get an address's neighbors in the address graph
                api_response = api_instance.list_address_neighbors(currency, address, direction)
                return api_response
            except graphsense.ApiException as e:
                print("Exception when calling AddressesApi->list_address_neighbors: %s\n" % e)

            # example passing only required values which don't have defaults set
            # and optional values
            #try:
                # Get an address's neighbors in the address graph
                #api_response = api_instance.list_address_neighbors(currency, address, direction, only_ids=only_ids, include_labels=include_labels, page=page, pagesize=pagesize)
                #pprint(api_response)
            #except graphsense.ApiException as e:
                #print("Exception when calling AddressesApi->list_address_neighbors: %s\n" % e)

    #Utility function to get neighbors of address        
    def get_addr_neighbors(self, address, direction):

        if (direction == 'out'):
            degree = 'out_degree'
            address_degree = 'address_out_degree'
        else:
            degree = 'in_degree'
            address_degree = 'address_in_degree'

        with graphsense.ApiClient(configuration) as api_client:
            try:
                api_instance = bulk_api.BulkApi(api_client)
                print(f"get_addr_neighbors of {address}")
                operation = "list_address_neighbors"
                body = {'address': [address], 'direction': direction}

                df_address_neighbors = pd.read_csv(api_instance.bulk_csv(self.CURRENCY, operation, body=body,
                                                                    num_pages=1, _preload_content=False))
                
                df_address_neighbors = df_address_neighbors \
                    .loc[(df_address_neighbors['_error'] != 'not found') &
                        (df_address_neighbors['_info'] != 'no data')].reset_index(drop=True)
                
                if df_address_neighbors.empty:
                    df_address_neighbors.columns = ['address', 'entity', degree]
                    return df_address_neighbors
                
                print(df_address_neighbors.columns)  # Print column names

                
                df_address_neighbors = df_address_neighbors \
                    .rename(columns={'address_address': 'address', 
                                    'address_entity': 'entity',
                                    address_degree: degree})

                return df_address_neighbors[['address', 'entity', degree]]
                
            except graphsense.ApiException as e:
                print("Exception when calling Bulk Api: %s\n" % e)

    # The Breadth-First Search algorithm:
    def bfs_neighbors(self, seed_address, max_depth, max_degree, verbose, direction):

        if (direction == 'out'):
            degree = 'out_degree'
            #address_degree = 'address_out_degree'
        else:
            degree = 'in_degree'
            #address_degree = 'address_in_degree'
        
        # collect neighbors
        neighbors = []
        #for i in range(0, max_depth):
            #neighbors.append([])
        neighbors.append([seed_address])
                
        # record visited addresses and entities
        visited_addresses = []
        
        # maintain a queue of paths
        queue = []
        
        # push the first path into the queue
        queue.append([seed_address])
        
        # count number of requests
        no_requests = 0
        
        while(queue):

            # get first path from the queue
            path = queue.pop(0)
            print(f"No requests: {no_requests}, " +
                f"Queue size: {len(queue)}, " +
                f"path length: {len(path)}, " +
                f"seed address {seed_address}", end='\r')
            
            # get the last node from the path
            addr = path[-1]

            # retrieve address neighbors
            df_neighbors = self.get_addr_neighbors(addr, direction)
            no_requests = no_requests + 1

            #All addresses from that level
            level = []

            # continue with neighbors out_degree < max_outdegree
            for index, neighbor in df_neighbors.iterrows():

                new_path = list(path)
                new_path.append(neighbor['address'])

                # found path (address match)
                #if(neighbor['address'] in target_addresses):
                    #print(new_path, end=' ')
                    #print("MATCH | address")
                    #matching_paths.append(new_path)
                    #continue

                # found path (entity match)
                #if(neighbor['entity'] in target_entities):
                    #print(new_path, end=' ')
                    #print("MATCH | entity")
                    #matching_paths.append(new_path)
                    #continue
                                    
                # stop if max depth is reached
                if len(new_path) == max_depth:
                    if verbose:
                        print(new_path, end=' ') 
                        print("STOP | max depth")
                    continue
                
                # stop if address has already been visited
                if(neighbor['address'] in visited_addresses):
                    if verbose:
                        print(new_path, end=' ') 
                        print("STOP | same address")
                    continue

                #Append the current address to the level array
                level.append(neighbor['address'])

                # stop if address out_degree exceeds threshold
                if(neighbor[degree] > max_degree):
                    if verbose:
                        print(new_path, end=' ') 
                        print("STOP | max degree")
                    continue

                queue.append(new_path)
                visited_addresses.append(neighbor['address'])

            neighbors.append(level)
                    
            if len(queue) == 0:
                return neighbors

# Blacklist Module

In [340]:
class Blacklist():

    def __init__(self, currency):
        self.CURRENCY = currency

    def ofac(self):

        coins = []
        myCoins = {}
        sdn = defaultdict(list)
        filename = 'sdn2.xml'
        URL = "https://www.treasury.gov/ofac/downloads/sanctions/1.0/sdn_advanced.xml"

        response = requests.get(URL)
        with open(filename, 'wb') as file:
            file.write(response.content)

        xml_data = open(filename, 'r').read()  # Read file
        d = xmltodict.parse(xml_data)

        for i in d['Sanctions']['ReferenceValueSets']['FeatureTypeValues']['FeatureType']:
            if 'Digital Currency Address' in i['#text']:
                coins.append(i['@ID'])
                myCoins[i['@ID']] =  i['#text'].replace('Digital Currency Address - ','')
            #   print(i['@ID'],'-',i['#text'])

        for i in d['Sanctions']['DistinctParties']['DistinctParty']:
            if 'Feature' in i['Profile'].keys():
                for j in i['Profile']['Feature']:
                    if '@FeatureTypeID' in j:
                        if type(j) is not str:
                            if str(j['@FeatureTypeID']) in coins:
                            #   print(j)
                            #   print(j['FeatureVersion']['VersionDetail']['#text'])
                                sdn[myCoins[j['@FeatureTypeID']]].append(j['FeatureVersion']['VersionDetail']['#text'])            
                            #    break
            
        with open('sdn.json', 'w') as fp:
            json_data = json.dump(sdn, fp)
        fp.close()

# Main Section

In [341]:
# Function calls

ass_wallets = AssociatedWallets(CURRENCY)

direction = 'in'

addr_neigh = ass_wallets.bfs_neighbors(ADDRESS, 3, 20, True, direction)
#addr_simple_neigh = ass_wallets.simple_neighbors(ADDRESS, direction)

with open("results/addresses_" + direction + ".txt", "w") as outfile:
    outfile.write(str(addr_neigh))

#blacklist = Blacklist(CURRENCY)
#blacklist.ofac()

get_addr_neighbors of 0x49bd56B275395130B41D67E1304b2d49F8A88725bd56B275395130B41D67E1304b2d49F8A88725
Index(['_error', '_info', '_request_address', 'address_actors',
       'address_actors_count', 'address_address', 'address_balance_eur',
       'address_balance_usd', 'address_balance_value', 'address_currency',
       'address_entity', 'address_first_tx_height',
       'address_first_tx_timestamp', 'address_first_tx_tx_hash',
       'address_in_degree', 'address_is_contract', 'address_last_tx_height',
       'address_last_tx_timestamp', 'address_last_tx_tx_hash',
       'address_no_incoming_txs', 'address_no_outgoing_txs',
       'address_out_degree', 'address_status', 'address_token_balances',
       'address_token_balances_usdc_eur', 'address_token_balances_usdc_usd',
       'address_token_balances_usdc_value', 'address_token_balances_usdt_eur',
       'address_token_balances_usdt_usd', 'address_token_balances_usdt_value',
       'address_token_balances_weth_eur', 'address_token_ba